In [12]:
#!pip install imgaug tqdm importlib

import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
import utils
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm
from importlib import reload
import pandas as pd
from shutil import copyfile
from time import time
import pickle

In [13]:
TRAIN_PATH = 'TrainData/images/'
TRAIN_AUGMENTED_PATH = 'TrainData/images_augmented/'
ANNOTATIONS_PATH = 'TrainData/annotations/'
ANNOTATIONS_AUGMENTED_PATH = 'TrainData/annotations_augmented/'
TEST_PATH = 'TestData/images/'

labelMeta = ['filepath', 'xmin', 'ymin', 'xmax', 'ymax', 'class_name']

In [14]:
!nvidia-smi

Tue Jul 30 17:22:41 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:07.0 Off |                    0 |
| N/A   36C    P0    36W / 250W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:00:08.0 Off |                    0 |
| N/A   36C    P0    36W / 250W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [15]:
count_epoch = 800
save_file = 'model_frcnn.hdf5'
interim_epoch = '2v100model_frcnn-epoch'

In [ ]:
for asdf in range(20):
    out = []
    
    for f in tqdm(os.listdir(TRAIN_PATH)):
        if f[-4:] != '.jpg':
            continue
        img = utils.read_image(TRAIN_PATH + f)
        img_boxes = BoundingBoxesOnImage(
            utils.get_bounding_boxes(ANNOTATIONS_PATH + f[:-4] + '.xml'), 
            shape=img.shape
        )
        img_aug, img_boxes_aug = utils.augment(img, img_boxes)
        utils.write_image(TRAIN_AUGMENTED_PATH + f, img_aug)
        out += utils.get_training_annotations(TRAIN_AUGMENTED_PATH + f, img_boxes_aug)
        utils.write_annotation_xml('images_augmented', ANNOTATIONS_AUGMENTED_PATH, f, img_aug, img_boxes_aug)

    with open('annotate_augmented.txt', 'w') as f:
        f.write('\n'.join(out))
    
    data = pd.read_csv('annotate_augmented.txt', sep=",", header=None, names=labelMeta)
    labels = pd.read_csv('annotate.txt', sep=",", header=None, names=labelMeta)
    
    labels2 = pd.concat([labels, data])
    
    labels2.to_csv('annotate_full_train.txt', header=None, index=None)
    
    !python train_frcnn_tensorboard.py -o simple --input_weight_path model_frcnn.hdf5 --hf --num_epochs=10 -p annotate_full_train.txt
    
    count_epoch+=10
    
    copyfile(save_file, interim_epoch+str(count_epoch)+'.hdf5')
    
    

100%|██████████| 178/178 [00:40<00:00,  7.86it/s]


Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0, 'ripe_strawberry': 2485, 'unripe_strawberry': 3644}
Num classes (including bg) = 3
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 437
Num val samples 76
W0730 07:38:07.063843 140147058280192 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 07:38:07.077749 140147058280192 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 07:38:07.082168 140147058280192 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4076: The name tf.random_uniform is deprecated. Please use t

In [ ]:
for asdf in range(2):
    out = []
    
    for f in tqdm(os.listdir(TRAIN_PATH)):
        if f[-4:] != '.jpg':
            continue
        img = utils.read_image(TRAIN_PATH + f)
        img_boxes = BoundingBoxesOnImage(
            utils.get_bounding_boxes(ANNOTATIONS_PATH + f[:-4] + '.xml'), 
            shape=img.shape
        )
        img_aug, img_boxes_aug = utils.augment(img, img_boxes)
        utils.write_image(TRAIN_AUGMENTED_PATH + f, img_aug)
        out += utils.get_training_annotations(TRAIN_AUGMENTED_PATH + f, img_boxes_aug)
        utils.write_annotation_xml('images_augmented', ANNOTATIONS_AUGMENTED_PATH, f, img_aug, img_boxes_aug)

    with open('annotate_augmented.txt', 'w') as f:
        f.write('\n'.join(out))
    
    data = pd.read_csv('annotate_augmented.txt', sep=",", header=None, names=labelMeta)
    labels = pd.read_csv('annotate.txt', sep=",", header=None, names=labelMeta)
    
    labels2 = pd.concat([labels, data])
    
    labels2.to_csv('annotate_full_train.txt', header=None, index=None)
    
    !python train_frcnn.py -o simple --input_weight_path model_frcnn.hdf5 --hf --num_epochs=25 -p annotate_full_train.txt
    
    count_epoch+=25
    
    copyfile(save_file, interim_epoch+str(count_epoch)+'.hdf5')

100%|██████████| 178/178 [00:38<00:00,  6.40it/s]


Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0, 'ripe_strawberry': 2485, 'unripe_strawberry': 3644}
Num classes (including bg) = 3
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 439
Num val samples 74
W0724 21:38:19.303321 140017237313280 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0724 21:38:19.316742 140017237313280 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0724 21:38:19.321135 140017237313280 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4076: The name tf.random_uniform is deprecated. Please use t

In [ ]:
for asdf in range(12):
    out = []
    
    for f in tqdm(os.listdir(TRAIN_PATH)):
        if f[-4:] != '.jpg':
            continue
        img = utils.read_image(TRAIN_PATH + f)
        img_boxes = BoundingBoxesOnImage(
            utils.get_bounding_boxes(ANNOTATIONS_PATH + f[:-4] + '.xml'), 
            shape=img.shape
        )
        img_aug, img_boxes_aug = utils.augment(img, img_boxes)
        utils.write_image(TRAIN_AUGMENTED_PATH + f, img_aug)
        out += utils.get_training_annotations(TRAIN_AUGMENTED_PATH + f, img_boxes_aug)
        utils.write_annotation_xml('images_augmented', ANNOTATIONS_AUGMENTED_PATH, f, img_aug, img_boxes_aug)

    with open('annotate_augmented.txt', 'w') as f:
        f.write('\n'.join(out))
    
    data = pd.read_csv('annotate_augmented.txt', sep=",", header=None, names=labelMeta)
    labels = pd.read_csv('annotate.txt', sep=",", header=None, names=labelMeta)
    
    labels2 = pd.concat([labels, data])
    
    labels2.to_csv('annotate_full_train.txt', header=None, index=None)
    
    !python train_frcnn.py -o simple --input_weight_path model_frcnn.hdf5 --hf --num_epochs=25 -p annotate_full_train.txt
    
    count_epoch+=25
    
    copyfile(save_file, interim_epoch+str(count_epoch)+'.hdf5')

100%|██████████| 178/178 [00:38<00:00,  8.29it/s]


Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0, 'ripe_strawberry': 2485, 'unripe_strawberry': 3644}
Num classes (including bg) = 3
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 422
Num val samples 91
W0725 15:20:57.174113 140592417687296 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 15:20:57.187801 140592417687296 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 15:20:57.192247 140592417687296 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4076: The name tf.random_uniform is deprecated. Please use t

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000/1000 [==============================] - 1108s 1s/step - rpn_cls: 0.3041 - rpn_regr: 0.0734 - detector_cls: 0.0796 - detector_regr: 0.0227
Mean number of bounding boxes from RPN overlapping ground truth boxes: 29.583
Classifier accuracy for bounding boxes from RPN: 0.9709375
Loss RPN classifier: 0.30410908091516226
Loss RPN regression: 0.0734124112528807
Loss Detector classifier: 0.07959875644849103
Loss Detector regression: 0.022718308863171843
Elapsed time: 1108.4811656475067
Epoch 19/25
Average number of overlapping bounding boxes from RPN = 29.583 for 1000 previous iterations
1000/1000 [==============================] - 1136s 1s/step - rpn_cls: 0.2776 - rpn_regr: 0.0708 - detector_cls: 0.0735 - detector_regr: 0.0219
Mean number of bounding boxes from RPN overlapping ground truth boxes: 30.932
Classifier accuracy for bounding boxes from RPN: 0.97121875
Loss RPN classifier: 0.2776288993649009
Loss RPN regression: 0.07079459297138965
Loss Detector classifier: 0.07352338564156304
L

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

1000/1000 [==============================] - 1145s 1s/step - rpn_cls: 0.3296 - rpn_regr: 0.0731 - detector_cls: 0.0830 - detector_regr: 0.0245
Mean number of bounding boxes from RPN overlapping ground truth boxes: 30.176
Classifier accuracy for bounding boxes from RPN: 0.9681875
Loss RPN classifier: 0.3295670767091673
Loss RPN regression: 0.07313303943630307
Loss Detector classifier: 0.08297392552447445
Loss Detector regression: 0.024477233809593598
Elapsed time: 1145.1340117454529
Epoch 13/25
Average number of overlapping bounding boxes from RPN = 30.176 for 1000 previous iterations
1000/1000 [==============================] - 1138s 1s/step - rpn_cls: 0.2968 - rpn_regr: 0.0692 - detector_cls: 0.0789 - detector_regr: 0.0233
Mean number of bounding boxes from RPN overlapping ground truth boxes: 30.393
Classifier accuracy for bounding boxes from RPN: 0.96925
Loss RPN classifier: 0.2968376729202712
Loss RPN regression: 0.06918970258205082
Loss Detector classifier: 0.07888552223848097
Loss

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000/1000 [==============================] - 1140s 1s/step - rpn_cls: 0.2801 - rpn_regr: 0.0636 - detector_cls: 0.0709 - detector_regr: 0.0217
Mean number of bounding boxes from RPN overlapping ground truth boxes: 30.349
Classifier accuracy for bounding boxes from RPN: 0.972875
Loss RPN classifier: 0.28010154536278914
Loss RPN regression: 0.06359684079307772
Loss Detector classifier: 0.07089135098667892
Loss Detector regression: 0.021658468132009147
Elapsed time: 1140.1661295890808
Epoch 21/25
Average number of overlapping bounding boxes from RPN = 30.349 for 1000 previous iterations
1000/1000 [==============================] - 1124s 1s/step - rpn_cls: 0.2481 - rpn_regr: 0.0583 - detector_cls: 0.0642 - detector_regr: 0.0184
Mean number of bounding boxes from RPN overlapping ground truth boxes: 30.723
Classifier accuracy for bounding boxes from RPN: 0.975375
Loss RPN classifier: 0.24809525597193471
Loss RPN regression: 0.05830138416429691
Loss Detector classifier: 0.06417363090963756
Lo

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000/1000 [==============================] - 1124s 1s/step - rpn_cls: 0.2395 - rpn_regr: 0.0577 - detector_cls: 0.0650 - detector_regr: 0.0188
Mean number of bounding boxes from RPN overlapping ground truth boxes: 29.819
Classifier accuracy for bounding boxes from RPN: 0.97478125
Loss RPN classifier: 0.239534662237914
Loss RPN regression: 0.057682109350454994
Loss Detector classifier: 0.0649618606612118
Loss Detector regression: 0.018821597399248276
Elapsed time: 1123.93039894104
Total loss decreased from 0.389000978123998 to 0.38100022964882907, saving weights
Epoch 24/25
Average number of overlapping bounding boxes from RPN = 29.819 for 1000 previous iterations
1000/1000 [==============================] - 1172s 1s/step - rpn_cls: 0.2342 - rpn_regr: 0.0580 - detector_cls: 0.0653 - detector_regr: 0.0189
Mean number of bounding boxes from RPN overlapping ground truth boxes: 30.099
Classifier accuracy for bounding boxes from RPN: 0.97490625
Loss RPN classifier: 0.23416295386109143
Loss R

In [ ]:
for _ in range(10):
    out = []
    
    # tqdm is misbehaving, reverting to caveman timing
    t = time()
    print('Augmenting and saving images...')
#     for f in tqdm(os.listdir(TRAIN_PATH)):
    for f in os.listdir(TRAIN_PATH):
        if f[-4:] != '.jpg':
            continue
        img = utils.read_image(TRAIN_PATH + f)
        img_boxes = BoundingBoxesOnImage(
            utils.get_bounding_boxes(ANNOTATIONS_PATH + f[:-4] + '.xml'), 
            shape=img.shape
        )
        img_aug, img_boxes_aug = utils.augment(img, img_boxes)
        utils.write_image(TRAIN_AUGMENTED_PATH + f, img_aug)
        out += utils.get_training_annotations(TRAIN_AUGMENTED_PATH + f, img_boxes_aug)
        utils.write_annotation_xml('images_augmented', ANNOTATIONS_AUGMENTED_PATH, f, img_aug, img_boxes_aug)

    with open('annotate_augmented.txt', 'w') as f:
        f.write('\n'.join(out))
    
    data = pd.read_csv('annotate_augmented.txt', sep=",", header=None, names=labelMeta)
    labels = pd.read_csv('annotate.txt', sep=",", header=None, names=labelMeta)
    
    labels2 = pd.concat([labels, data])
    
    labels2.to_csv('annotate_full_train.txt', header=None, index=None)
    
    print('...took {} seconds'.format(time() - t))
    
    !python train_frcnn_tensorboard.py -o simple --input_weight_path model_frcnn.hdf5 --hf --num_epochs=25 -p annotate_full_train.txt
    
    count_epoch+=25
    
    copyfile(save_file, interim_epoch+str(count_epoch)+'.hdf5')

Augmenting and saving images...
...took 38.831846714019775 seconds
Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0, 'ripe_strawberry': 2485, 'unripe_strawberry': 3644}
Num classes (including bg) = 3
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 426
Num val samples 87
W0730 04:10:53.669059 139980287723264 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 04:10:53.682790 139980287723264 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 04:10:53.687204 139980287723264 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_back

In [ ]:

print("Epoch: {}".format(count_epoch))
for _ in range(10):
    out = []
    
    # tqdm is misbehaving, reverting to caveman timing
    t = time()
    print('Augmenting and saving images...')
#     for f in tqdm(os.listdir(TRAIN_PATH)):
    for f in os.listdir(TRAIN_PATH):
        if f[-4:] != '.jpg':
            continue
        img = utils.read_image(TRAIN_PATH + f)
        img_boxes = BoundingBoxesOnImage(
            utils.get_bounding_boxes(ANNOTATIONS_PATH + f[:-4] + '.xml'), 
            shape=img.shape
        )
        img_aug, img_boxes_aug = utils.augment(img, img_boxes)
        utils.write_image(TRAIN_AUGMENTED_PATH + f, img_aug)
        out += utils.get_training_annotations(TRAIN_AUGMENTED_PATH + f, img_boxes_aug)
        utils.write_annotation_xml('images_augmented', ANNOTATIONS_AUGMENTED_PATH, f, img_aug, img_boxes_aug)

    with open('annotate_augmented.txt', 'w') as f:
        f.write('\n'.join(out))
    
    data = pd.read_csv('annotate_augmented.txt', sep=",", header=None, names=labelMeta)
    labels = pd.read_csv('annotate.txt', sep=",", header=None, names=labelMeta)
    
    labels2 = pd.concat([labels, data])
    
    labels2.to_csv('annotate_full_train.txt', header=None, index=None)
    
    print('...took {} seconds'.format(time() - t))
    
    with open('curent_epoch.pickle','wb') as f:
        pickle.dump(count_epoch, f)
    
    !python train_frcnn_tensorboard.py -o simple --ce=curent_epoch.pickle --input_weight_path=model_frcnn.hdf5 --hf --num_epochs=25 -p annotate_full_train.txt
    
    count_epoch+=25
    
    copyfile(save_file, interim_epoch+str(count_epoch)+'.hdf5')

Epoch: 800
Augmenting and saving images...
...took 37.216914653778076 seconds
Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0, 'ripe_strawberry': 2485, 'unripe_strawberry': 3644}
Num classes (including bg) = 3
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 422
Num val samples 91
W0730 17:24:03.370579 140425556768512 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 17:24:03.384079 140425556768512 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 17:24:03.388494 140425556768512 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tens

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 900/1000 [==========================>...] - ETA: 1:49 - rpn_cls: 0.1946 - rpn_regr: 0.0460 - detector_cls: 0.0557 - detector_regr: 0.0146On step 900 of epoch 18.  Writing these losses to Tensorboard...
RPN Losses:  {'loss': 0.95632267, 'rpn_out_class_loss': 0.9101126, 'rpn_out_regress_loss': 0.046210073}
Classifier Losses:  {'loss': 0.95632267, 'dense_class_3_loss': 0.9101126, 'dense_regress_3_loss': 0.046210073}
1000/1000 [==============================] - 1096s 1s/step - rpn_cls: 0.2024 - rpn_regr: 0.0461 - detector_cls: 0.0545 - detector_regr: 0.0147
Mean number of bounding boxes from RPN overlapping ground truth boxes: 31.337
Classifier accuracy for bounding boxes from RPN: 0.9801875
Loss RPN classifier: 0.2024327913719394
Loss RPN regression: 0.04613682368319132
Loss Detector classifier: 0.05450419495281542
Loss Detector regression: 0.014653079146250093
Elapsed time: 1096.4731392860413
Total loss decreased from 0.32910560222078244 to 0.3177268891541962, saving weights
Epoch 20/25

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 900/1000 [==========================>...] - ETA: 1:55 - rpn_cls: 0.4192 - rpn_regr: 0.0869 - detector_cls: 0.1250 - detector_regr: 0.0391On step 825900 of epoch 0.  Writing these losses to Tensorboard...
RPN Losses:  {'loss': 0.011938746, 'rpn_out_class_loss': 1.0380263e-06, 'rpn_out_regress_loss': 0.011937708}
Classifier Losses:  {'loss': 0.011938746, 'dense_class_3_loss': 1.0380263e-06, 'dense_regress_3_loss': 0.011937708}
1000/1000 [==============================] - 1156s 1s/step - rpn_cls: 0.4149 - rpn_regr: 0.0873 - detector_cls: 0.1233 - detector_regr: 0.0385
Mean number of bounding boxes from RPN overlapping ground truth boxes: 29.78
Classifier accuracy for bounding boxes from RPN: 0.9545
Loss RPN classifier: 0.4148888313291255
Loss RPN regression: 0.08731928830611287
Loss Detector classifier: 0.12327900259156649
Loss Detector regression: 0.038452710621146254
Elapsed time: 1157.8917319774628
Total loss decreased from inf to 0.6639398328479511, saving weights
Epoch 2/25
Average 

In [30]:
count_epoch=0
new_save_file = '2v100RestartedTraining_Epoch'

with open('curent_epoch.pickle','wb') as f:
    pickle.dump(count_epoch, f)
    
!python train_frcnn_tensorboard.py -o simple --ce=curent_epoch.pickle --hf --num_epochs=25 -p annotate_full_train.txt
count_epoch+=25
    
copyfile(save_file, new_save_file+str(count_epoch)+'.hdf5')

Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0, 'ripe_strawberry': 2485, 'unripe_strawberry': 3644}
Num classes (including bg) = 3
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 430
Num val samples 83
W0804 15:44:07.398761 140535666280192 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 15:44:07.412354 140535666280192 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 15:44:07.416801 140535666280192 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4076: The name tf.random_uniform is deprecated. Please use t

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 500/1000 [==============>...............] - ETA: 9:23 - rpn_cls: 2.6653 - rpn_regr: 0.4014 - detector_cls: 0.3868 - detector_regr: 0.2361On step 7500 of epoch 7.  Writing these losses to Tensorboard...
RPN Losses:  {'loss': 1.864961, 'rpn_out_class_loss': 1.5692905, 'rpn_out_regress_loss': 0.29567048}
Classifier Losses:  {'loss': 1.864961, 'dense_class_3_loss': 1.5692905, 'dense_regress_3_loss': 0.29567048}
 600/1000 [=================>............] - ETA: 7:35 - rpn_cls: 2.6660 - rpn_regr: 0.4051 - detector_cls: 0.3850 - detector_regr: 0.2334On step 7600 of epoch 7.  Writing these losses to Tensorboard...
RPN Losses:  {'loss': 5.1032352, 'rpn_out_class_loss': 4.956679, 'rpn_out_regress_loss': 0.14655614}
Classifier Losses:  {'loss': 5.1032352, 'dense_class_3_loss': 4.956679, 'dense_regress_3_loss': 0.14655614}
 700/1000 [====================>.........] - ETA: 5:39 - rpn_cls: 2.6642 - rpn_regr: 0.4047 - detector_cls: 0.3925 - detector_regr: 0.2355On step 7700 of epoch 7.  Writing thes

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 800/1000 [=======================>......] - ETA: 3:38 - rpn_cls: 2.1002 - rpn_regr: 0.3360 - detector_cls: 0.3545 - detector_regr: 0.1986On step 14800 of epoch 14.  Writing these losses to Tensorboard...
RPN Losses:  {'loss': 3.2481148, 'rpn_out_class_loss': 3.1088595, 'rpn_out_regress_loss': 0.1392552}
Classifier Losses:  {'loss': 3.2481148, 'dense_class_3_loss': 3.1088595, 'dense_regress_3_loss': 0.1392552}
 900/1000 [==========================>...] - ETA: 1:50 - rpn_cls: 2.1160 - rpn_regr: 0.3364 - detector_cls: 0.3507 - detector_regr: 0.1952On step 14900 of epoch 14.  Writing these losses to Tensorboard...
RPN Losses:  {'loss': 5.7469597, 'rpn_out_class_loss': 4.5015597, 'rpn_out_regress_loss': 1.2453997}
Classifier Losses:  {'loss': 5.7469597, 'dense_class_3_loss': 4.5015597, 'dense_regress_3_loss': 1.2453997}
1000/1000 [==============================] - 1103s 1s/step - rpn_cls: 2.0781 - rpn_regr: 0.3342 - detector_cls: 0.3509 - detector_regr: 0.1951
Mean number of bounding boxes

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 500/1000 [==============>...............] - ETA: 9:01 - rpn_cls: 1.6487 - rpn_regr: 0.2715 - detector_cls: 0.3105 - detector_regr: 0.1542On step 21500 of epoch 21.  Writing these losses to Tensorboard...
RPN Losses:  {'loss': 0.35077932, 'rpn_out_class_loss': 0.13194877, 'rpn_out_regress_loss': 0.21883056}
Classifier Losses:  {'loss': 0.35077932, 'dense_class_3_loss': 0.13194877, 'dense_regress_3_loss': 0.21883056}
 600/1000 [=================>............] - ETA: 7:16 - rpn_cls: 1.6650 - rpn_regr: 0.2791 - detector_cls: 0.3077 - detector_regr: 0.1525On step 21600 of epoch 21.  Writing these losses to Tensorboard...
RPN Losses:  {'loss': 6.6463976, 'rpn_out_class_loss': 5.8965726, 'rpn_out_regress_loss': 0.74982506}
Classifier Losses:  {'loss': 6.6463976, 'dense_class_3_loss': 5.8965726, 'dense_regress_3_loss': 0.74982506}
 700/1000 [====================>.........] - ETA: 5:28 - rpn_cls: 1.6779 - rpn_regr: 0.2821 - detector_cls: 0.3085 - detector_regr: 0.1535On step 21700 of epoch 21.

'2v100RestartedTraining_Epoch25.hdf5'

In [37]:
count_epoch = 40

In [38]:
print("Epoch: {}".format(count_epoch))
for _ in range(15):
    out = []
    
    # tqdm is misbehaving, reverting to caveman timing
    t = time()
    print('Augmenting and saving images...')
#     for f in tqdm(os.listdir(TRAIN_PATH)):
    for f in os.listdir(TRAIN_PATH):
        if f[-4:] != '.jpg':
            continue
        img = utils.read_image(TRAIN_PATH + f)
        img_boxes = BoundingBoxesOnImage(
            utils.get_bounding_boxes(ANNOTATIONS_PATH + f[:-4] + '.xml'), 
            shape=img.shape
        )
        img_aug, img_boxes_aug = utils.augment(img, img_boxes)
        utils.write_image(TRAIN_AUGMENTED_PATH + f, img_aug)
        out += utils.get_training_annotations(TRAIN_AUGMENTED_PATH + f, img_boxes_aug)
        utils.write_annotation_xml('images_augmented', ANNOTATIONS_AUGMENTED_PATH, f, img_aug, img_boxes_aug)

    with open('annotate_augmented.txt', 'w') as f:
        f.write('\n'.join(out))
    
    data = pd.read_csv('annotate_augmented.txt', sep=",", header=None, names=labelMeta)
    labels = pd.read_csv('annotate.txt', sep=",", header=None, names=labelMeta)
    
    labels2 = pd.concat([labels, data])
    
    labels2.to_csv('annotate_full_train.txt', header=None, index=None)
    
    print('...took {} seconds'.format(time() - t))
    
    with open('curent_epoch.pickle','wb') as f:
        pickle.dump(count_epoch, f)
    
    !python train_frcnn_tensorboard.py -o simple --ce=curent_epoch.pickle --input_weight_path=model_frcnn.hdf5 --hf --num_epochs=25 -p annotate_full_train.txt
    
    count_epoch+=25
    
    copyfile(save_file, new_save_file+str(count_epoch)+'.hdf5')

Epoch: 40
Augmenting and saving images...
...took 38.620166540145874 seconds
Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0, 'ripe_strawberry': 2485, 'unripe_strawberry': 3644}
Num classes (including bg) = 3
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 427
Num val samples 86
W0805 05:29:05.173717 140393299011328 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 05:29:05.187309 140393299011328 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 05:29:05.191769 140393299011328 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tenso

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...took 35.201887130737305 seconds
Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0, 'ripe_strawberry': 2485, 'unripe_strawberry': 3644}
Num classes (including bg) = 3
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 433
Num val samples 80
W0805 23:46:58.944490 140351870564096 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 23:46:58.957965 140351870564096 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 23:46:58.962436 140351870564096 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4076: The name tf.random_

KeyboardInterrupt: 